**Text Summarization using BART Transformer**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [ ]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

without fine -tuning

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

pipe = pipeline("summarization" , model ="facebook/bart-large-cnn")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
article_1 = ds['train'][1]['dialogue']

In [ ]:
pipe(article_1, max_length=20, min_length=10, do_sample = False)

[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots.'}]

with fine - tuning

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [ ]:
# tokenization
def preprocess_function(batch):
    source = batch["dialogue"]
    target = batch["summary"]
    source_ids = tokenizer(source, padding="max_length", truncation=True, max_length = 12 )
    target_ids = tokenizer(source, padding="max_length", truncation=True, max_length = 12 )

    labels = target_ids["input_ids"]
    labels = [[(label if label != tokenizer.pad_token_id else - 100) for label in labels_example] for labels_example in labels]

    return{
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels,
    }

In [ ]:
df_source = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# training_arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "/content",
    num_train_epochs = 2,
    remove_unused_columns = True,
    per_device_train_batch_size = 8,
    report_to="none"  # Disable all integrations, including wandb
)


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = df_source["train"],
    eval_dataset = df_source["test"]
)

In [ ]:
!pip uninstall wandh -y

In [ ]:
trainer.train()

Step,Training Loss
500,0.044000
1000,0.023400
1500,0.017300
2000,0.009900
2500,0.004700
3000,0.002800


TrainOutput(global_step=3116, training_loss=0.016450977854344597, metrics={'train_runtime': 1180.4283, 'train_samples_per_second': 21.111, 'train_steps_per_second': 2.64, 'total_flos': 632862265835520.0, 'train_loss': 0.016450977854344597, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
eval_results

{'eval_loss': 0.2699419856071472,
 'eval_runtime': 10.8228,
 'eval_samples_per_second': 138.596,
 'eval_steps_per_second': 17.371,
 'epoch': 2.0}

saving the model

In [ ]:
model.save_pretrained("/content/model_directory")
tokenizer.save_pretrained("/content/model_directory")

('/content/model_directory/tokenizer_config.json',
 '/content/model_directory/special_tokens_map.json',
 '/content/model_directory/vocab.json',
 '/content/model_directory/merges.txt',
 '/content/model_directory/added_tokens.json',
 '/content/model_directory/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/model_directory")

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
def summarize(blog_post):
    inputs = tokenizer(blog_post, return_tensors="pt", max_length=1024, truncation=True)

    summary_ids = model.generate(inputs['input_ids'],max_length = 150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    # decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
blog_post = """
An autoencoder is a special type of neural network that learns to encode an input sentence into lower dimensional representations and decode
the embedded representations back to the corresponding original input sentences. In a general case, when the input and output sentence of an
autoencoder is the same, over a large number of iterations, the autoencoder network directly maps the input token to the output tokens, and
the embedded representation that is usually learned between them becomes redundant. Therefore, we modify the input sentence by randomly
deleting word tokens and replacing them with a special <MASK> token, this sentence with the randomly deleted token is called a corrupted or
noisy sentence and the supervised output for the corresponding input is the clean sentence with all the original tokens preserved.
By learning to predict the missing or corrupted tokens, the denoising autoencoder learns to extract meaningful features from the input
sentence. A denoising autoencoder is trained on a large corpus of such data so it learns to predict the masked/deleted token in the input
sentence which is responsible for the noise in the text, as a result, we get a clean and semantically coherent output, hence the term
“denoising” is added to the autoencoder.
"""
summary = summarize(blog_post)
print( f'summary : {summary}')


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


summary : #Person1# is a special type of such data so it learns to predict the noise in the text, as a result, we get a clean sentence preserved.
#Joe.
#
